In [2]:
## For cleaning GA Data. Remember that the goal here is to: 
## (1) Turn our policy results into something the machine algorithm can easily read
## (2) Sort the individual motions into their appropriate categories. This will help us later discuss the implications of our machine's predictions.
## (3) Delete data used for stress testing - such as the meeting number.  

### IMPORTANT: You must run the code in the order presented. 

import numpy as np 
import pandas as pd

import matplotlib
import matplotlib.pyplot as pp

import csv

In [4]:
#Reads and displays the Data file
dfGA = pd.read_csv('GA_Query_CleanLIWC')
dfGA

,Unnamed: 0,date,Class M,Class S,Class I,Class P,Class B,Policy Passed,Conflict Indicator,WC,Analytic,Clout,Authentic,Tone,WPS,Sixltr,Dic,function,pronoun,ppron,i,we,you,shehe,they,ipron,article,prep,auxverb,adverb,conj,negate,verb,adj,compare,interrog,number,quant,affect,posemo,...,health,sexual,ingest,drives,affiliation,achieve,power,reward,risk,focuspast,focuspresent,focusfuture,relativ,motion,space,time,work,leisure,home,money,relig,death,informal,swear,netspeak,assent,nonflu,filler,AllPunc,Period,Comma,Colon,SemiC,QMark,Exclam,Dash,Quote,Apostro,Parenth,OtherP
0,0,2012,1,0,0,0,0,0,1,20075.0,99.00,68.60,14.49,96.27,27.24,34.34,77.56,42.78,5.44,2.15,0.35,1.48,0.00,0.07,0.24,3.29,11.21,16.88,3.82,1.51,5.00,0.26,6.50,6.69,1.68,0.35,2.43,1.01,7.11,5.98,...,0.60,0.0,0.25,11.52,3.10,2.91,3.37,1.12,2.16,1.37,4.82,0.97,12.55,1.22,8.62,2.69,6.46,0.50,0.09,0.54,0.12,0.03,0.22,0.0,0.10,0.01,0.10,0.0,11.80,4.04,4.34,0.03,0.04,0.00,0.0,1.23,0.07,0.64,0.82,0.60
1,1,2012,0,0,3,0,0,0,0,822.0,99.00,74.84,17.74,41.15,20.05,26.64,72.38,43.19,5.23,2.92,1.46,1.09,0.24,0.00,0.12,2.31,12.77,16.55,5.35,1.70,2.19,0.36,10.71,2.43,1.09,0.73,5.72,1.46,1.70,1.22,...,0.00,0.0,0.12,8.64,2.31,1.09,4.38,1.09,0.49,1.95,6.08,1.70,12.90,1.34,7.54,4.26,5.84,0.12,0.12,0.00,0.00,0.00,0.36,0.0,0.36,0.00,0.00,0.0,23.60,11.56,3.04,1.70,0.00,0.00,0.0,0.85,0.49,0.12,4.38,1.46
2,2,2003,0,0,0,0,0,0,0,314.0,99.00,66.07,4.02,31.19,19.62,29.30,72.93,38.54,2.55,0.32,0.00,0.00,0.00,0.32,0.00,2.23,13.38,17.20,4.14,0.32,0.96,0.00,6.37,3.82,1.27,0.00,13.06,1.27,0.64,0.32,...,0.00,0.0,0.96,8.92,0.32,0.96,7.32,0.32,0.00,4.46,2.55,0.32,10.83,0.64,6.05,3.82,10.51,0.00,0.32,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,34.08,20.70,3.50,0.96,0.00,0.00,0.0,2.23,0.00,0.64,3.18,2.87
3,3,1995,0,0,0,0,0,1,1,17021.0,98.45,62.93,24.54,58.00,32.30,30.04,75.49,46.31,6.64,2.17,0.66,0.95,0.01,0.09,0.45,4.48,11.69,16.96,5.22,2.20,4.48,0.57,7.80,4.04,1.46,0.79,4.00,1.33,4.85,3.25,...,0.17,0.0,0.09,7.50,2.27,1.74,2.61,0.88,0.79,1.59,5.12,1.11,12.84,0.95,8.61,3.24,4.57,0.39,0.08,0.42,0.09,0.09,0.35,0.0,0.26,0.03,0.06,0.0,13.34,3.44,4.91,0.25,0.17,0.02,0.0,1.33,0.22,0.16,0.64,2.18
4,4,2007,0,0,0,0,0,0,0,9289.0,98.94,60.45,8.85,70.17,22.82,32.95,78.23,44.38,4.40,0.82,0.02,0.00,0.00,0.58,0.22,3.58,13.27,15.19,5.80,1.71,4.61,0.56,9.08,3.90,1.47,0.44,4.94,1.35,3.93,3.07,...,0.52,0.0,0.00,9.01,1.17,3.18,3.89,0.72,0.70,5.12,2.73,0.86,11.68,1.35,7.12,3.12,7.94,0.17,0.36,1.98,0.00,0.04,0.08,0.0,0.01,0.03,0.03,0.0,14.38,5.37,3.80,0.16,0.15,0.00,0.0,0.93,0.28,0.75,1.42,1.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10207,10207,2004,0,0,0,0,0,0,0,4059.0,98.95,58.85,3.46,71.55,23.46,31.44,75.41,45.31,4.73,0.74,0.00,0.00,0.00,0.62,0.12,3.99,12.59,16.38,6.16,1.58,4.07,1.01,9.04,4.09,1.21,0.64,5.15,1.13,4.51,3.40,...,0.64,0.0,0.02,7.71,1.43,1.90,3.45,0.49,1.03,4.88,2.76,0.49,9.83,0.64,6.01,3.13,6.43,0.10,0.05,2.12,0.00,0.07,0.05,0.0,0.00,0.00,0.02,0.0,14.46,5.57,3.72,0.15,0.12,0.00,0.0,1.18,0.00,0.76,1.72,1.23
10208,10208,1994,0,0,0,0,0,0,0,8210.0,99.00,56.93,10.94,62.72,25.42,30.78,77.17,45.35,4.31,0.57,0.06,0.00,0.00,0.30,0.21,3.74,13.01,17.15,5.57,1.49,4.71,0.35,7.94,3.67,1.80,0.88,5.31,1.32,3.02,2.46,...,0.24,0.0,0.13,7.10,0.71,2.02,3.25,0.94,0.55,4.95,2.36,0.82,12.30,0.94,7.00,4.26,8.36,0.22,0.06,3.74,0.07,0.05,0.12,0.0,0.01,0.00,0.11,0.0,12.63,4.53,3.58,0.12,0.22,0.00,0.0,1.06,0.02,0.29,1.05,1.75
10209,10209,2013,0,0,0,0,0,0,0,583.0,99.00,67.21,10.22,31.63,27.76,25.39,71.70,43.91,6.52,1.54,0.86,0.00,0.17,0.17,0.34,4.97,13.72,15.95,3.95,1.37,3.60,0.00,7.20,2.74,0.69,0.17,10.46,1.54,0.51,0.34,...,0.00,0.0,0.00,7.38,1.03,0.34,3.43,2.40,0.17,2.23,4.63,1.54,10.29,0.51,5.49,3.43,7.55,0.00,0.17,1.03,0.17,0.00,0.17,0.0,0.17,0.00,0.00,0.

In [5]:
# Next, we're going to sort the motions by their categories. 

dfGA['Class M'] = dfGA['May I take it that the assembly wishes to take note of those items that remain open for consideration'] 
+ dfGA['The agenda was adopted'] 
+ dfGA['I have been authorized to make the following statement on behalf of the assembly'] 
+ dfGA['Refer the matter']
+ dfGA['Report of the Committee']
+ dfGA['The Assembly will now take a decision on draft resolution']
+ dfGA['Appoint a Committee']

dfGA['Class S'] = dfGA['Amend the agenda to'] 
+ dfGA['To adjourn the debate'] 
+ dfGA['May I take it that it is the wish of the General Assembly to conclude its consideration']
+ dfGA['To introduce a draft amendment to the draft resolution']

dfGA['Class I'] = dfGA['Point of Order']
+ dfGA['Right of reply']
+ dfGA['Objection to consideration of the question']
+ dfGA['To withdraw']
+ dfGA['A recorded vote has been requested']
+ dfGA['A paragraph-by-paragraph vote on the draft resolution']

dfGA['Class P'] = dfGA['To suspend the meeting'] + dfGA['To adjourn the meeting']

dfGA['Class B'] = dfGA['Reconsideration of the']

dfGA['Policy Passed'] = dfGA['The draft resolution was adopted']

dfGA['Conflict Indicator'] = dfGA['Sanctions'] + dfGA['Peacekeeping'] + dfGA['Peacemaking']

dfGA

KeyError: ignored

In [ ]:
#Delete columns of queries not needed and now summarized. 
#Deleted columns include: motion queries, meeting records for stress testing

dfGA.drop(['Unnamed: 0', 
           'May I take it that the assembly wishes to take note of those items that remain open for consideration', 
           'The agenda was adopted', 
           'Amend the agenda to', 
           'I have been authorized to make the following statement on behalf of the assembly',
           'Refer the matter',
           'Report of the Committee',
           'Point of Order',
           'Right of reply',
           'To adjourn the debate',
           'May I take it that it is the wish of the General Assembly to conclude its consideration',
           'To suspend the meeting',
           'To adjourn the meeting',
           'To introduce a draft amendment to the draft resolution',
           'The Assembly will now take a decision on draft resolution',
           'Objection to consideration of the question',
           'To withdraw',
           'Reconsideration of the',
           'Appoint a Committee',
           'A recorded vote has been requested',
           'A paragraph-by-paragraph vote on the draft resolution',
           'The draft resolution was adopted',
           'Sanctions',
           'Peacekeeping',
           'Peacemaking'], axis = 1, inplace= True)

dfGA

In [ ]:
#Transform the Policy Passed Label and Conflict Indicators into a binary for classification
# Do not keep as a frequency - query stress tested for meeting passing policy, not the measurement of that policy (some meetings have 50+ passed policies on technical issues)
# This is necessary to properly run Sigmoid activation function. If you want to keep policy counts, you will need to change 
#the label activation function

a = np.array(dfGA['Policy Passed'].values.tolist())
print(a)
dfGA['Policy Passed'] = np.where(a > 1, 1, a).tolist()

b = np.array(dfGA['Conflict Indicator'].values.tolist())
print(b)
dfGA['Conflict Indicator'] = np.where(b > 1, 1, b).tolist()

dfGA.head(20)

In [ ]:
#Some replicators separately parsed the date. If you did, uncomment
#If your webscrapper also parsed separately the merge code will help. Otherwise, ignore and just use the date function to properly summarize years 

#date = pd.read_csv('record_dates.csv')

#date['record'] = date['record'].map(lambda x: str(x)[:-4])

#dfGA1 = dfGA.merge(date, left_on = "meeting", right_on = "record")

#dfGA1["date"] = dfGA1["date"].map(lambda x: x if len(x) == 4 else x.split("/")[0])
#dfGA1

In [ ]:
LIWC_GA = pd.read_csv('GA_LIWC.csv')

LIWC_GA

In [ ]:
#Merge with LIWC 

LIWC_GA = pd.read_csv('GA_LIWC.csv')

LIWC_GA['Filename'] = LIWC_GA['Filename'].map(lambda x: str(x)[:-4]) ### EDIT

dfGA1 = dfGA.merge(LIWC_GA, left_on = "meeting", right_on = "Filename")

dfGA1["meeting"] = dfGA1["meeting"].map(lambda x: x if len(x) == 4 else x.split("/")[0])
dfGA1

In [ ]:
#Final Cleaning
# WARNING: change to dfGA1 if you have needed to combine separate files. 

#dfGA1.drop([''], axis = 1, inplace= True) #Specify any additional columns created to delete - usually 'unnamed'

dfGA1.drop(['meeting', 
           'Filename', 
           'Segment'], axis = 1, inplace= True)

dfGA1

In [ ]:
dfGA2 = pd.to_numeric(dfGA1['Class M'], downcast = 'float')

dfGA2

In [ ]:
dfGA1.to_csv('GA_Query_Clean')